# PRFPY plotter example

How to do plotting from prfpy outputs

In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
import os
from dag_prf_utils.prfpy_ts_plotter import TSPlotter


In [ ]:
# What is the index of the output of prfpy
# When fitting with prfpy - the output is a numpy array of shape
# n voxels x n parameters

# I translate them to there named versions using this: 

def prfpy_params_dict():
    '''
    Easy look up table for prfpy model parameters
    name to index...
    '''
    p_order = {}
    # [1] gauss. Note hrf_1, and hrf_2 are idx 5 and 6, if fit...
    p_order['gauss'] = {
        'x'             :  0, # mu_x
        'y'             :  1, # mu_y
        'size_1'        :  2, # size
        'amp_1'         :  3, # beta
        'bold_baseline' :  4, # baseline 
        'hrf_deriv'     :  5, # *hrf_1
        'hrf_disp'      :  6, # *hrf_2
        'rsq'           : -1, # ... 
    }    
    # [2] css. Note hrf_1, and hrf_2 are idx 6 and 7, if fit...
    p_order['css'] = {
        'x'             :  0, # mu_x
        'y'             :  1, # mu_y
        'size_1'        :  2, # size
        'amp_1'         :  3, # beta
        'bold_baseline' :  4, # baseline 
        'n_exp'         :  5, # n
        'hrf_deriv'     :  6, # *hrf_1
        'hrf_disp'      :  7, # *hrf_2        
        'rsq'           : -1, # ... 
    }

    # [3] dog. Note hrf_1, and hrf_2 are idx 7 and 8, if fit...
    p_order['dog'] = {
        'x'             :  0, # mu_x
        'y'             :  1, # mu_y
        'size_1'        :  2, # prf_size
        'amp_1'         :  3, # prf_amplitude
        'bold_baseline' :  4, # bold_baseline 
        'amp_2'         :  5, # srf_amplitude
        'size_2'        :  6, # srf_size
        'hrf_deriv'     :  7, # *hrf_1
        'hrf_disp'      :  8, # *hrf_2        
        'rsq'           : -1, # ... 
    }

    p_order['norm'] = {
        'x'             :  0, # mu_x
        'y'             :  1, # mu_y
        'size_1'        :  2, # prf_size
        'amp_1'         :  3, # prf_amplitude
        'bold_baseline' :  4, # bold_baseline 
        'amp_2'         :  5, # srf_amplitude
        'size_2'        :  6, # srf_size
        'b_val'         :  7, # neural_baseline 
        'd_val'         :  8, # surround_baseline
        'hrf_deriv'     :  9, # *hrf_1
        'hrf_disp'      : 10, # *hrf_2        
        'rsq'           : -1, # rsq
    }            

    p_order['csf']  ={
        'width_r'       : 0,
        'SFp'           : 1,
        'CSp'          : 2,
        'width_l'       : 3,
        'crf_exp'       : 4,
        'amp_1'         : 5,
        'bold_baseline' : 6,
        'hrf_1'         : 7,
        'hrf_2'         : 8,
        'rsq'           : -1,
    }

    return p_order


In [ ]:
# Quick tutorial on what is possible:
prf_params = ... # Load the numpy array you get out of prfpy 
# -> i.e., fitter.iterative_search_params 
prfpy_model = ... # prfpy.model object (which contains the design matrix). You needed this to do the fitting
# e.g., 
prfpy_model = prfpy.model.Iso2DGaussianModel(
    ... # whatever you put in here for the fitting
)

# Load the real_ts (i.e. the data)
real_ts = ... # nvoxes x time points

prf_plotter = TSPlotter(
    prf_params=prf_params,             # Put the numpy array you get out of prfpy 
    model = 'gauss',            # name of the model type 'gauss', 'norm', 'dog', 'css' are options
    real_ts = real_ts,              # numpy array of the data used to do the fitting
    prfpy_model = prfpy_model,
)

In [ ]:
# Now what can you do?
# want to look at the parameters, now in pd.DataFrame, with useful labels!
print(prf_plotter.pd_params)


In [ ]:
# Want to find which voxels fit some criteria? i.e., only look at good fits?
binary_mask = prf_plotter.return_vx_mask(th={
    'min-rsq' : 0.1, # only include fits with rsq > 0.1
    'max-ecc' : 5, # only include fits where the location is inside a certain eccentricity    
})
# You can apply any threshold to any parameter in .pd_params 
# Options include:
#       min,max,bound,eq

print(f'Total number of voxels in binary mask = {binary_mask.sum()}')
print(f'Out of a total of {binary_mask.shape[0]}')


In [ ]:
# Some plotting stuff:
# -> visual field
prf_plotter.visual_field(
    th={'min-rsq': 0.1}, # only plot voxels with good rsq
    dot_col='rsq', # color the dots by the rsq 
    dot_vmin=0, dot_vmax=1, cmap='viridis', # set colorbar properties
    dot_size=10, # control the size of the dots
)

In [ ]:
# More plotting stuff
# -> histograms: specify any parameter in .pd_params, and any threshold...
prf_plotter.hist(
    param='x', th={'min-rsq':.1} , 
)

In [ ]:
# More plotting stuff
# -> scatter plots: scatter any parameter with any other... 
prf_plotter.scatter(
    px='ecc', py='size_1', th={'min-rsq':.1}
)

In [ ]:
# -> plot time course
# plot the time series for voxel with index=0
# will plot the data; the prediction; also the receptive field
# it should also plot the parameter values somewhere
_ = prf_plotter.prf_ts_plot(idx=0)